## Adquisición y preprocesamiento de la información - RafaC

<table>
    <tr>
      <td>Grado en Ingeniería de Datos e Inteligencia Artificial - Facultad de Informática - UCM
      </td>
      <td>
      <img src="https://biblioteca.ucm.es/data/cont/media/www/pag-88746//escudo.jpg"  width=50/>
      </td>
     </tr>
</table>




### Análisis de textos

El mundo del análisis de textos es totalmente diferente a lo visto hasta ahora, con sus propios problemas y bibliotecas especializadas. Vamos a ver solo algunas de sus características.

Empecemos asegurándonos de que las bibliotecas necesarias están instaladas


In [49]:
modules = ["wikipedia==1.4.0",  "bs4", "spacy==3.4.1", "nltk"]


import sys
import os.path
from subprocess import check_call
import importlib
import os

def instala(modules):
  print("Instalando módulos")
  for m in modules:
      # para el import quitamos [...] y ==...
      p = m.find("[")
      mi = m if p==-1 else m[:p]
      p = mi.find("==")
      mi = mi if p==-1 else mi[:p]

      torch_loader = importlib.util.find_spec(mi)
      if torch_loader is not None:
          print(m," encontrado")
      else:
          print(m," No encontrado, instalando...",end="")  
          try:        
            r = check_call([sys.executable, "-m", "pip", "install", "--user", m])
            print("¡hecho!")
          except:
            print("¡Problema al instalar ",m,"! ¿seguro que el módulo existe?",sep="")
              
  print("¡Terminado!")

instala(modules) 

Instalando módulos
wikipedia==1.4.0  encontrado
bs4  encontrado
spacy==3.4.1  encontrado
nltk  encontrado
¡Terminado!


## Tokens

La primera idea es obtener las piezas sintácticas básicas, palabras, símbolos de puntuación, etc., pero esto no es tan trivial como parece. Empezamos usando la función estándar `split`

In [50]:
s = "Desde lo alto se divisa la ciudad y toda la campiña"
l = s.split(" ")
l

['Desde',
 'lo',
 'alto',
 'se',
 'divisa',
 'la',
 'ciudad',
 'y',
 'toda',
 'la',
 'campiña']

In [51]:
s = "Se remueve, levanta una tenue polvareda, avanza."
l = s.split(" ")
l

['Se', 'remueve,', 'levanta', 'una', 'tenue', 'polvareda,', 'avanza.']

Bibliotecas como spaCy permiten obtener mejor los tokens

In [52]:
import spacy
nlp = spacy.blank('es') # modelo
s = "Se remueve, levanta una tenue polvareda, avanza."
doc =  nlp(s)
l = [token.text for token in doc]
l

['Se',
 'remueve',
 ',',
 'levanta',
 'una',
 'tenue',
 'polvareda',
 ',',
 'avanza',
 '.']

Podemos "limpiar" el texto quedando solo con un tipo de palabras

In [53]:
l =  [token.text for token in doc if  token.is_alpha] 
l

['Se', 'remueve', 'levanta', 'una', 'tenue', 'polvareda', 'avanza']

Un caso particular de *token* son las stop words, palabras sin semántica que pueden desvirtuar nuestros análisis

In [54]:
l =  [token.text for token in doc if  token.is_alpha and not token.is_stop] 
l

['remueve', 'levanta', 'tenue', 'polvareda', 'avanza']

### Contando oraciones y palabras

Igual que con las palabras podemos intentarlo desde Python estándar:

In [55]:
s = "he comprado patatas, naranjas, manzanas..."
punto = s.find(".") # posición del primer punto
if punto!=-1: # -1 si no hay puntos
  print("La primera oración: ", s[:punto])
  print("El resto del texto: ", s[punto+1:])

La primera oración:  he comprado patatas, naranjas, manzanas
El resto del texto:  ..


In [56]:
!python -m spacy download es_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     --------------------------------------- 12.9/12.9 MB 15.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


DEPRECATION: https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.4.0/es_core_news_sm-3.4.0-py3-none-any.whl#egg=es_core_news_sm==3.4.0 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617

[notice] A new release of pip is available: 23.0.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [57]:
import es_core_news_sm
nlp = es_core_news_sm.load()
s = "he comprado patatas, naranjas, manzanas... todo en \
     http://verdurasfrescas.com. ¡Y ha llegado muy rápido!"
doc = nlp(s)
for i,o in enumerate(doc.sents):
    print(i+1,o)


1 he comprado patatas, naranjas, manzanas... todo en      http://verdurasfrescas.com.
2 ¡Y ha llegado muy rápido!


In [58]:
from bs4 import BeautifulSoup
import requests
pagina = "https://www.gutenberg.org/cache/epub/8870/pg8870.html"
page = requests.get(pagina)
soup = BeautifulSoup(page.text, 'html.parser')  # le pasamos el texto en HTML para que lo analice
soup.text

'\nThe Project Gutenberg eBook of La Mejor Cocinera, Recetas de Cocina, by Calleja\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nThe Project Gutenberg eBook of La Mejor Cocinera, Recetas de Cocina\n\r\nThis ebook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this ebook or online at www.gutenberg.org. If you are not located in the United States, you’ll have to check the laws of the country where you are located before using this eBook.\n\nTitle: La Mejor Cocinera, Recetas de Cocina\n\nAuthor: Calleja\n\nRelease date: September 1, 2005 [eBook #8870]Most recently updated: September 13, 2014\nLanguage: Spanish\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK LA MEJOR COCINERA, RECETAS DE COCINA ***\n\nProduced by Distributed Proofreaders\nLA MEJOR COCINERA pará CALLEJA\n RECETAS DE COCINA\nÍNDICE\nA LAS S

In [63]:
doc =  nlp(soup.text)
print("El libro incluye ", len(list(doc.sents)), "oraciones")

El libro incluye  1826 oraciones


Si lo que queremos es contar palabras podemos utilizar `Counter` que incluye un diccionario con cada letra de la palabra y su frecuencia. 

In [64]:
from collections import Counter
palabras =  [token.text for token in doc if  token.is_alpha and not token.is_stop] 
c = Counter(palabras)

In [65]:
c

Counter({'Project': 85,
         'Gutenberg': 85,
         'eBook': 10,
         'of': 112,
         'Cocinera': 3,
         'Recetas': 3,
         'Cocina': 3,
         'by': 26,
         'Calleja': 2,
         'This': 3,
         'ebook': 2,
         'is': 22,
         'for': 22,
         'use': 9,
         'anyone': 5,
         'anywhere': 2,
         'in': 58,
         'United': 15,
         'States': 15,
         'and': 65,
         'most': 4,
         'other': 16,
         'parts': 2,
         'world': 2,
         'at': 13,
         'cost': 3,
         'with': 46,
         'almost': 2,
         'restrictions': 2,
         'whatsoever': 2,
         'You': 12,
         'may': 15,
         'copy': 12,
         'it': 13,
         'give': 3,
         'away': 2,
         'or': 71,
         'under': 5,
         'terms': 20,
         'License': 10,
         'included': 3,
         'this': 39,
         'online': 4,
         'If': 16,
         'you': 55,
         'are': 23,
         'not':

In [66]:
c.most_common(20)

[('sal', 331),
 ('agua', 325),
 ('pone', 321),
 ('manteca', 274),
 ('azúcar', 225),
 ('aceite', 202),
 ('echa', 193),
 ('horno', 189),
 ('salsa', 181),
 ('fuego', 178),
 ('huevo', 171),
 ('harina', 167),
 ('cebolla', 154),
 ('huevos', 152),
 ('perejil', 144),
 ('ponen', 144),
 ('gramos', 140),
 ('caldo', 139),
 ('pan', 122),
 ('punto', 121)]

Podemos añadir "the" como stopword

In [67]:
nlp = spacy.blank("es")
nlp.Defaults.stop_words.add("the")
doc =  nlp(soup.text)
palabras =  [token.text for token in doc if  token.is_alpha and not token.is_stop] 
c = Counter(palabras)
c.most_common(20)

[('sal', 331),
 ('agua', 325),
 ('pone', 321),
 ('manteca', 274),
 ('azúcar', 225),
 ('aceite', 202),
 ('echa', 193),
 ('horno', 189),
 ('salsa', 181),
 ('fuego', 178),
 ('huevo', 171),
 ('harina', 167),
 ('cebolla', 154),
 ('huevos', 152),
 ('perejil', 144),
 ('ponen', 144),
 ('gramos', 140),
 ('caldo', 139),
 ('pan', 122),
 ('punto', 121)]

### Lemas y raíces

El resultado anterior se ve afectado por el hecho de que la misma palabra puede aparecer en singular, plural, verbos conjugados de distinta forma, etc.

In [41]:
import es_core_news_sm
nlp = es_core_news_sm.load()
doc =  nlp(soup.text)
palabras = [p.lemma_.upper() for p in doc
            if p.is_alpha and not p.is_stop]
c = Counter(palabras)
c.most_common(20)

[('PONER', 517),
 ('AGUA', 342),
 ('SAL', 334),
 ('HUEVO', 325),
 ('COCER', 310),
 ('MANTECA', 266),
 ('ECHAR', 265),
 ('AZÚCAR', 228),
 ('SERVIR', 210),
 ('ACEITE', 210),
 ('SALSA', 198),
 ('HORNO', 189),
 ('FUEGO', 181),
 ('DEJAR', 176),
 ('HARINA', 158),
 ('CEBOLLA', 155),
 ('YEMA', 151),
 ('CALDO', 149),
 ('PEREJIL', 144),
 ('GRAMO', 143)]

La **extracción de raíces**, conocida en la literatura en inglés como *stemming*, que convierte palabras a su raíz eliminando sufijos de persona, género, etc. Es una alternativa a la lematización que ha sido tradicionalmente utilizada en aplicaciones de inteligencia artificial, por ejemplo en *aprendizaje automático*, pero que no es tan útil para nuestros propósitos al no obtenerse palabras completas como resultados

In [42]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('spanish')
palabras = [stemmer.stem(p.text).upper() for p in doc
              if p.is_alpha and not p.is_stop]
frecuencia = Counter(palabras)
palabras_comunes = frecuencia.most_common(5)

palabras_comunes

[('PON', 556), ('HUEV', 364), ('AGU', 343), ('SAL', 341), ('COC', 328)]

In [47]:
from nltk.stem import SnowballStemmer
import es_core_news_sm
nlp = es_core_news_sm.load()  
doc = nlp(soup.text) 

stemmer = SnowballStemmer('spanish')

palabras = [p.text.upper() for p in doc        
             if p.is_alpha and not p.is_stop]  
print("Palabras no vacías en el texto ", len(set(palabras)))   

palabras = [p.lemma_.upper() for p in doc
            if p.is_alpha and not p.is_stop]
print("Con lemas ", len(set(palabras)))   
palabras = [stemmer.stem(p.text).upper() for p in doc
              if p.is_alpha and not p.is_stop]
print("Con raíces ", len(set(palabras)))   

Palabras no vacías en el texto  3889
Con lemas  3428
Con raíces  2485
